In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import polars as pl
import geopandas as gpd 
import numpy as np
import re
import time

import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'u'
project = 'Ciencia de los datos'
data = make_dir_line(modality, project)

raw = data('raw')
processed = data('processed')

In [ ]:
file_path = raw / 'DCIS_POPRES1 - Resident population on 1st January - full dataset.csv'
chunk_size = 1000000  # Cantidad de filas por fragmento
parts = 500
i = 0
re_express = re.compile('(\d{1,3})( years)(.*)')
dfs = []
dfs2 = []

In [ ]:
# df = pd.read_csv(raw / 'DCIS_POPRES1 - Resident population on 1st January - full dataset.csv', 
#                  sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
# df.head()

In [ ]:
# Codigo de dia 1

# # Iterar sobre los fragmentos
# i = 0
# for chunk in pd.read_csv(file_path, chunksize=chunk_size, sep = '|', decimal = '.', header = 0, encoding = 'utf-8'):
#     if i == parts + 1:
#         print('parte: ', i)
#         break
#     else:
#         chunk = chunk[(chunk['TIME'] == 2022) & (chunk['SEXISTAT1'] != 9) 
#                     & (chunk['STATCIV2'] != 99) & (chunk['Age'] != 'total')]
        
#         ter = chunk.copy()
#         ter = ter.loc[:,['ITTER107', 'Territory']]
#         ter['ITTER107'] = ter['ITTER107'].astype(str)
        
#         chunk = chunk.loc[:,['ITTER107', 'SEXISTAT1', 'Age', 'STATCIV2', 'Value']]
#         chunk['Age'] = chunk['Age'].str.replace(re_express, r'\1', regex=True)
#         chunk['ITTER107'] = chunk['ITTER107'].astype(str)
#         chunk['Age'] = chunk['Age'].astype(int)
#         # meter en una lista
#         dfs.append(chunk)
#         dfs2.append(ter)
#         i = i + 1
#         if i%5==0:
#             print(i) 
# # concatenar y guardar
# print('concatenando')
# chunk = pd.concat(dfs, ignore_index=True)
# ter = pd.concat(dfs2, ignore_index=True)
# ter = ter.drop_duplicates()
# print('saving')
# chunk.to_parquet(processed / 'chunk.parquet.gzip', compression='gzip')
# ter.to_parquet(processed / 'ter.parquet.gzip', compression='gzip')

In [ ]:
# Iterar sobre los fragmentos
for chunk in pd.read_csv(file_path, chunksize=chunk_size, sep = '|', decimal = '.', header = 0, encoding = 'utf-8'):
    if i == parts + 1:
        print('parte: ', i)
        break
    else:
        chunk = chunk[(chunk['TIME'] == 2022) & (chunk['SEXISTAT1'] != 9) 
                    & (chunk['STATCIV2'] != 99) & (chunk['Age'] != 'total')]
        
        ter = chunk.copy()
        ter = ter.loc[:,['ITTER107', 'Territory']]
        ter['ITTER107'] = ter['ITTER107'].astype(str)
        ter['Territory'] = ter['Territory'].str.title()
                
        chunk = chunk.loc[:,['ITTER107', 'SEXISTAT1', 'Age', 'STATCIV2', 'Value']]
        chunk['Age'] = chunk['Age'].str.replace(re_express, r'\1', regex=True)
        chunk['ITTER107'] = chunk['ITTER107'].astype(str)
        chunk['Age'] = chunk['Age'].astype(int)
        
        # meter en una lista
        dfs.append(chunk)
        dfs2.append(ter)
        i = i + 1
        if i%5==0:
            print(i) 
            # concatenar y guardar
            print('concatenando')
            chunk = pd.concat(dfs, ignore_index=True)
            ter = pd.concat(dfs2, ignore_index=True)
            ter = ter.drop_duplicates()
            print('saving')
            chunk.to_parquet(processed / 'chunk.parquet.gzip', compression='gzip')
            chunk.to_csv(processed / 'chunk.csv', encoding = 'utf-8-sig', index = False)
            ter.to_parquet(processed / 'ter.parquet.gzip', compression='gzip')
            ter.to_csv(processed / 'ter.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
ter['Territory'] = ter['Territory'].str.replace('Centro (I)','Centro')
ter['Territory'] = ter['Territory'].str.replace('Valle D\'Aosta / Vallée D\'Aoste','Valle d\'Aosta')
ter['Territory'] = ter['Territory'].str.replace('Trentino Alto Adige / Südtirol','Trentino-Alto Adige')
ter['Territory'] = ter['Territory'].str.replace('Friuli-Venezia Giulia','Friuli Venezia Giulia')
ter.to_parquet(processed / 'ter.parquet.gzip', compression='gzip')
ter.to_csv(processed / 'ter.csv', encoding = 'utf-8-sig', index = False)

SEXISTAT1

1- Male   <br>
2- Female <br>
9- Total  <br>

STATCIV2
1- never married <br>
2- married       <br>
3- divorced      <br>
4- widowed       <br>
99- total        <br>

columns of italy <br>
Index(['ITTER107', 'Territory', 'TIPO_DATO15', 'Demographic data type',
       'SEXISTAT1', 'Gender', 'ETA1', 'Age', 'STATCIV2', 'Marital status',
       'TIME', 'Select time', 'Value', 'Flag Codes', 'Flags'],
      dtype='object')

In [ ]:
print('Ok_')